

# ⭐️ Give This Project a Star!

If you find this tool helpful, please consider giving it a **star** on GitHub! 🌟  
Also, don’t forget to **follow me** for more awesome projects! 🙌


[![GitHub Stars](https://img.shields.io/github/stars/codermillat/AudioMasterTool?style=social)](https://github.com/codermillat/AudioMasterTool/stargazers)  
[![GitHub Follow](https://img.shields.io/github/followers/codermillat?style=social)](https://github.com/codermillat)





In [ ]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Input Form

Mode = "Splitting" #@param ["Splitting"]
dataset = "Youtube" #@param ["Youtube", "Drive"]
url = "" #@param {type:"string"}
drive_path = "" #@param {type:"string"} #video path
AUDIO_NAME = "" #@param {type:"string"}

In [ ]:
#@title 1. Install Libraries
%pip install yt_dlp
%pip install ffmpeg
%pip install -U demucs
%pip install pydub
%pip install numpy librosa soundfile

import os
import subprocess
import shutil
import yt_dlp
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import librosa
import soundfile as sf
import numpy as np

# Create directories
os.makedirs('audio_input', exist_ok=True)
os.makedirs('audio_output', exist_ok=True)
os.makedirs('dataset', exist_ok=True)

# Clear directories
def clear_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

# Clear output directories
clear_directory('audio_input')
clear_directory('audio_output')
clear_directory('dataset')

In [ ]:
#@title 2. Download Audio
if dataset == "Drive":
  print("Dataset is set to Drive. Skipping YouTube section.")
elif dataset == "Youtube":
  !mkdir youtubeaudio

  ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'wav',
      }],
      "outtmpl": f'youtubeaudio/{AUDIO_NAME}',
  }

  def download_from_url(url):
      with yt_dlp.YoutubeDL(ydl_opts) as ydl:
          ydl.download([url])

  download_from_url(url)
  AUDIO_INPUT = f"/content/youtubeaudio/{AUDIO_NAME}.wav"
  print("Downloaded YouTube audio.")

In [ ]:
#@title 3. Separate Vocal and Instrument/Noise using Demucs
if dataset == "Drive":
  command = f"demucs --two-stems=vocals {drive_path}"
elif dataset == "Youtube":
  command = f"demucs --two-stems=vocals {AUDIO_INPUT}"

result = subprocess.run(command.split(), stdout=subprocess.PIPE)
print(result.stdout.decode())

VOCAL_PATH = f"/content/separated/htdemucs/{AUDIO_NAME}/vocals.wav"
print("Separation complete.")

In [ ]:
#@title 4. Remove Silence from Vocals
def remove_silence(input_path, output_path):
    # Load the audio file
    audio = AudioSegment.from_file(input_path, format="wav")

    # Detect non-silent chunks
    non_silents = detect_nonsilent(audio, min_silence_len=500, silence_thresh=-40)

    # Combine non-silent chunks
    output_audio = AudioSegment.empty()
    for start, end in non_silents:
        output_audio += audio[start:end]

    # Export the processed audio file
    output_audio.export(output_path, format="wav")

VOCAL_CLEANED_PATH = f"/content/audio_output/{AUDIO_NAME}_cleaned.wav"
remove_silence(VOCAL_PATH, VOCAL_CLEANED_PATH)
print("Silence removed.")

In [ ]:
#@title 5. Split the Cleaned Audio into Smaller Chunks
class Slicer:
    def __init__(self,
                 sr: int,
                 threshold: float = -40.,
                 min_length: int = 5000,
                 min_interval: int = 300,
                 hop_size: int = 20,
                 max_sil_kept: int = 5000):
        if threshold > 0:
            # ensure the threshold is negative as it represents dB
            threshold *= -1
        self.threshold = 10 ** (threshold / 20.)
        self.hop_size = round(sr * hop_size / 1000)
        self.min_length = round(sr * min_length / 1000 / self.hop_size)
        self.min_interval = round(min_interval / self.hop_size)
        self.max_sil_kept = round(sr * max_sil_kept / 1000 / self.hop_size)

    def slice(self, waveform):
        samples = waveform if len(waveform.shape) == 1 else waveform.mean(axis=0)
        rms_list = librosa.feature.rms(y=samples, frame_length=2048, hop_length=512).squeeze()
        sil_tags = []
        silence_start = None
        clip_start = 0
        for i, rms in enumerate(rms_list):
            # check if the rms is below the threshold
            if rms < self.threshold:
                if silence_start is None:
                    silence_start = i
                continue
            # if not silent, check if there was a previous silence
            if silence_start is None:
                continue
            # check if the silence is long enough and the clip is long enough
            if silence_start > 0 and i - silence_start >= self.min_interval and i - clip_start >= self.min_length:
                sil_tags.append([clip_start, i])
                clip_start = i
            silence_start = None
        # add the last clip if it's long enough
        if i - clip_start >= self.min_length:
            sil_tags.append([clip_start, i])

        # extract the audio clips based on the silence tags
        chunks = []
        for start, end in sil_tags:
            chunks.append(waveform[:, start * 512: end * 512])
        return chunks

# Load the cleaned vocal audio
audio, sr = librosa.load(VOCAL_CLEANED_PATH, sr=None, mono=False)

# Initialize slicer
slicer = Slicer(sr=sr, threshold=-40, min_length=5000, min_interval=500, hop_size=10, max_sil_kept=500)

# Slice the audio
chunks = slicer.slice(audio)

# Ensure the directory exists
!mkdir -p /content/dataset/{AUDIO_NAME}

# Save sliced audio
for i, chunk in enumerate(chunks):
    chunk_filename = f'/content/dataset/{AUDIO_NAME}/split_{i}.wav'
    sf.write(chunk_filename, chunk.T, sr) # Transpose the chunk array

print(f"Audio has been split and saved in /content/dataset/{AUDIO_NAME}/")

In [ ]:
#@title 6. Copy Results to Google Drive
!mkdir -p /content/drive/MyDrive/audio/{AUDIO_NAME}
!mkdir -p /content/drive/MyDrive/dataset/{AUDIO_NAME}
!cp -r /content/separated/htdemucs/{AUDIO_NAME}/* /content/drive/MyDrive/audio/{AUDIO_NAME}
!cp -r /content/dataset/{AUDIO_NAME}/* /content/drive/MyDrive/dataset/{AUDIO_NAME}